In [3]:
import os
import pandas as pd
import numpy as np
import matplotlib as plt
import tensorflow as tf



print(os.getcwd())

/home/henry/Work/ML/league-win-predict


In [12]:
# Import data 
game_state_20 = pd.read_csv('data/full_data_20.csv')
game_state_40 = pd.read_csv('data/full_data_40.csv')
game_state_60 = pd.read_csv('data/full_data_60.csv')
game_state_80 = pd.read_csv('data/full_data_80.csv')
game_state_100 = pd.read_csv('data/full_data_100.csv')

game_states = [game_state_20, game_state_40, game_state_60, game_state_80, game_state_100]

print(game_state_20.iloc[0,:])

Unnamed: 0                              0
matchID                    BR1_2720891721
fullTimeMS                        2092233
timePercent                             6
blueChampionKill                        5
blueFirstBlood                       True
blueDragonKill                          0
blueDragonHextechKill                   0
blueDragonChemtechKill                  0
blueDragonFireKill                      0
blueDragonAirKill                       0
blueDragonEarthKill                     0
blueDragonWaterKill                     0
blueDragonElderKill                     0
blueRiftHeraldKill                      0
blueBaronKill                           0
blueTowerKill                           0
blueInhibitorKill                       0
blueTotalGold                        9299
blueMinionsKilled                      85
blueJungleMinionsKilled                24
blueAvgPlayerLevel                    4.6
blueWin                              True
redChampionKill                   